In [159]:
import Pkg; Pkg.add("HiGHS")
import Pkg; Pkg.add("DataFrames")
import Pkg; Pkg.add("CSV")
using JuMP
using HiGHS
using DataFrames
using CSV


Plants and markets

In [160]:
P_g=["umm_al_nar", "ruwais", "taweelah", "dub_shar"] #gas stations
P_s=["shams", "al_dafra"]  # solar stations
P_h = ["hatta"] #hydro station
P_n = ["barakah"] #nuclear station

P=["umm_al_nar", "ruwais", "taweelah", "dub_shar", "shams", "al_dafra", "hatta", "barakah"] 

M=["dubai", "abudhabi", "sharjah"] # markets for products

3-element Vector{String}:
 "dubai"
 "abudhabi"
 "sharjah"

In [161]:
demand = DataFrame(CSV.File("/Users/pranavmadhusudhana/Documents/Visual Studio 2022/PSP/demand_uae5.csv"))
desalination_demand = DataFrame(CSV.File("/Users/pranavmadhusudhana/Documents/Visual Studio 2022/PSP/desalination_demand_uae5.csv"))
new_desalination_demand = DataFrame(CSV.File("/Users/pranavmadhusudhana/Documents/Visual Studio 2022/PSP/new_desalination_demand.csv"))
generators = DataFrame(CSV.File("/Users/pranavmadhusudhana/Documents/Visual Studio 2022/PSP/Generators_data.csv"))
new_generators = DataFrame(CSV.File("/Users/pranavmadhusudhana/Documents/Visual Studio 2022/PSP/new_Generators_data.csv"))
branches = DataFrame(CSV.File("/Users/pranavmadhusudhana/Documents/Visual Studio 2022/PSP/branch.csv"))
nodes = DataFrame(CSV.File("/Users/pranavmadhusudhana/Documents/Visual Studio 2022/PSP/Nodes.csv"))
solar = DataFrame(CSV.File("/Users/pranavmadhusudhana/Documents/Visual Studio 2022/PSP/solar.csv"))
L = branches.id
G = generators.Name[1:(size(generators,1))]  # note we truncate hydro and solar for now
gen_node = generators.Node
H = demand.Hour[1:8760] #the hours that we are working with

8760-element Vector{Int64}:
    1
    2
    3
    4
    5
    6
    7
    8
    9
   10
    ⋮
 8752
 8753
 8754
 8755
 8756
 8757
 8758
 8759
 8760

In [162]:
markets = DataFrame(
    market=M,  
    demand=[dubai_demand, Abu_dhabi_demand, sharjah_demand]
)

UndefVarError: UndefVarError: `dubai_demand` not defined

plants_g = DataFrame(plant=P_g, capacity=[350,650,100,300], cap_exg = [100,100,100,100], crf = [1,1,1,1], fix_oc = [100,120,100,100], var_oc = [100,120,100,100], fc = [1,1,1,1])
plants_s = DataFrame(plant=P_s, capacity=[100,200],  cap_exg = [100,100], crf = [1,1], fix_oc = [100,100], var_oc = [100,120,100,100], fc = [1,1] )
plants_h = DataFrame(plant=P_h, capacity=[350], cap_exg = [100], crf = [1], fix_oc = [100], var_oc = [100,120,100,100], fc = [1])
plants_n = DataFrame(plant=P_n, capacity=[1000], cap_exg = [100], crf = [1], fix_oc = [200], var_oc = [100,120,100,100], fc = [1])
new_plants_s = DataFrame(plant=P_s, capacity=[100,200],  cap_exg = [100,100], crf = [1,1], fix_oc = [100,100], var_oc = [100,120,100,100], fc = [1,1] )
new_plants_g = DataFrame(plant=P_g, capacity=[350,650,100,300], cap_exg = [100,100,100,100], crf = [1,1,1,1], fix_oc = [100,120,100,100], var_oc = [100,120,100,100], fc = [1,1,1,1])

plants = DataFrame(plant=P, capacity=[350,650,100,300, 100,200, 350, 1000], cap_exg = [100,100,100,100, 100,100, 100, 100], crf = [1,1,1,1, 1,1,1,1], fix_oc = [100,120,100,100, 100,100, 100, 200], var_oc = [100,120,100,100, 100,120, 100,100], fc = [1,1,1,1,1,1,1,1])

plants_g = DataFrame(plant=P_g, capacity=[350,650,100,300], cap_exg = [100,100,100,100], crf = [1,1,1,1], fix_oc = [100,120,100,100], var_oc = [100,120,100,100], fc = [1,1,1,1])
plants_s = DataFrame(plant=P_s, capacity=[100,200],  cap_exg = [100,100], crf = [1,1], fix_oc = [100,100], var_oc = [100,120], fc = [1,1] )
plants_h = DataFrame(plant=P_h, capacity=[350], cap_exg = [100], crf = [1], fix_oc = [100], var_oc = [100], fc = [1])
plants_n = DataFrame(plant=P_n, capacity=[1000], cap_exg = [100], crf = [1], fix_oc = [200], var_oc = [100], fc = [1])
new_plants_s = DataFrame(plant=P_s, capacity=[100,200],  cap_exg = [100,100], crf = [1,1], fix_oc = [100,100], var_oc = [100,120], fc = [1,1] )
new_plants_g = DataFrame(plant=P_g, capacity=[350,650,100,300], cap_exg = [100,100,100,100], crf = [1,1,1,1], fix_oc = [100,120,100,100], var_oc = [100,120,100,100], fc = [1,1,1,1])



In [163]:
direction_flow = zeros(length(nodes.id), length(branches.id))
branch_id = branches.id;
nodes_id = nodes.id;
N = nodes_id;

In [164]:
for l in branch_id
    direction_flow[branches[l,"From"],l] = -1
    direction_flow[branches[l,"To"],l] = 1
end

transmission  = Model(HiGHS.Optimizer);

@variables(transmission , begin
        CAP[g in G] >=0          # Generating capacity built (MW)
        GEN[g in G, h in H] >= 0 # Generation in each hour (MWh)
        NSE[h in H] >= 0         # Non-served energy in each hour (MWh)
        flow[l in L, h in H]
        LIN[l in L]
        NEW_CAP[n in N]    
        NEW_T_CAP[l in L]          # new build transmission capacity (MW)
        NEW_GEN[n in N, h in H]
        Area[n in N]

end)

transmission_losses = 90 # Cost of freight shipment per unit of distance
solar_inv = 100
fc_solar = 10
operating_ccgt = 100
operating_solar = 100
transmission_inv = 1000
NSECost = 6000 #need to find an actual value for 

6000

In [165]:
solar[10,1]

56

In [166]:
demand_balance = @constraint(
    transmission, [n in nodes_id, h in H], 
    sum(direction_flow[n,l]*flow[l,h] for l in branch_id) + sum(GEN[g,h] for g in G[(generators.Node .== n),:]) + NEW_GEN[n,h] 
    == first(demand[h, n]) + first(desalination_demand[h, n]) + first(new_desalination_demand[h, n])
)

@constraints( transmission, begin
    solar_gen[n in nodes_id, h in H], solar[h,1]*0.25*Area[n]*new_generators[n, 7]<= new_generators[n, 6]
    area_bound[n in nodes_id], Area[n] <= 21000000
    generation_solar[n in nodes_id[(new_generators.solar .== 1),:], h in H], NEW_GEN[n, h]<= solar[h,1]*0.25*Area[n]
end
)

max_transmision = @constraint(transmission, [l in L,  h in H], flow[l,h] <= 506.8 + NEW_T_CAP[l])
min_transmision = @constraint(transmission, [l in L,  h in H], flow[l,h] >= - 506.8 - NEW_T_CAP[l])
new_desalination = @constraint(transmission, [n in nodes_id,  h in H], first(new_desalination_demand[h, n]) <= NEW_CAP[n])



@constraints(transmission, begin
new_generators_constraint[n in nodes_id,  h in H], new_generators[n, 6] >= NEW_CAP[n]
new_generators_constraint2[n in nodes_id,  h in H], NEW_CAP[n] >= NEW_GEN[n,h]
generators_constraint[n in nodes_id,  h in H, g in G], generators[n, 15] >= GEN[g,h]
overall_generator[n in nodes_id,  h in H, g in G], generators[n, 15] + NEW_CAP[n] >= GEN[g,h] + NEW_GEN[n,h]
end);



InterruptException: InterruptException:

In [167]:
@objective(transmission, Min, sum(sum(generators[generators.Name.==g,:Var_Cost][1]*GEN[g,h] for h in H)
    for g in G) + sum(NSECost*NSE[h] for h in H) 
        + sum(NEW_T_CAP[l]*branches[l,:cost_of_new]*branches[l,:Length] for l in L) + 
        sum(new_generators[new_generators.Node.==n,:Fixed_Cost][1]*NEW_CAP[n] for n in N) 
        + sum(sum(new_generators[new_generators.Node.==n,:Var_Cost][1]*NEW_GEN[n,h] for n in N) for h in H) 
);

#@objective(transmission, Min,
 #   sum(generators[generators.Name.==g,:Fixed_Cost][1]*CAP[g] + 
  #      sum(generators[generators.Name.==g,:Var_Cost][1]*GEN[g,h] for h in H)
   # for g in G) + 
    #sum(NSECost*NSE[h] for h in H) 
#);

optimize!(transmission)


In [ ]:
generation = zeros(size(G,1))
for i in 1:size(G,1) 
    generation[i] = sum(value.(GEN)[G[i],:].data) 
end

new_generation = zeros(size(N,1))
for i in 1:size(N,1) 
    new_generation[i] = sum(value.(NEW_GEN)[N[i],:].data) 
end
#MWh_share = generation./sum(demand.total).*100
#cap_share = value.(CAP).data./maximum(demand.total).*100

results2 = DataFrame(
    new_t_cap = value.(NEW_T_CAP).data,
)
results = DataFrame(
    Resource = N, 
    area = value.(Area).data,
    #Percent_MW = cap_share,
    new_MWh = new_generation,
    MWh = generation,
    #Percent_GWh = MWh_share
)

results3 = DataFrame(
    nse = value.(NSE).data,
    new_gen = value.(NEW_GEN)[1,:].data,

    #Percent_MW = cap_share,
    #Percent_GWh = MWh_share
)
results
results

